# 長文をそのままOpenSearchに登録し、Highlightに対してRAGをする


---
## OpenSearch 起動・停止

* 起動

In [1]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" up -d


 Network 0_opensearch-docker_opensearch-net  Creating
 Network 0_opensearch-docker_opensearch-net  Created
 Container opensearch-dashboards  Creating
 Container opensearch-node1  Creating
 Container opensearch-dashboards  Created
 Container opensearch-node1  Created
 Container opensearch-dashboards  Starting
 Container opensearch-node1  Starting
 Container opensearch-node1  Started
 Container opensearch-dashboards  Started


* 停止

In [93]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" down
# docker compose -f "../0_opensearch-docker/docker-compose.yml" down -v


 Container opensearch-node1  Stopping
 Container opensearch-dashboards  Stopping
 Container opensearch-dashboards  Stopped
 Container opensearch-dashboards  Removing
 Container opensearch-dashboards  Removed


In [2]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" logs


opensearch-dashboards  | Disabling OpenSearch Security Dashboards Plugin
opensearch-dashboards  | Removing securityDashboards...
opensearch-dashboards  | Plugin removal complete


opensearch-node1       | Disabling execution of install_demo_configuration.sh for OpenSearch Security Plugin
opensearch-node1       | Disabling OpenSearch Security Plugin
opensearch-node1       | Enabling execution of OPENSEARCH_HOME/bin/opensearch-performance-analyzer/performance-analyzer-agent-cli for OpenSearch Performance Analyzer Plugin
opensearch-node1       | WARNING: A terminally deprecated method in java.lang.System has been called
opensearch-node1       | WARNING: System::setSecurityManager has been called by org.opensearch.bootstrap.OpenSearch (file:/usr/share/opensearch/lib/opensearch-2.11.0.jar)
opensearch-node1       | WARNING: Please consider reporting this to the maintainers of org.opensearch.bootstrap.OpenSearch
opensearch-node1       | WARNING: System::setSecurityManager will be removed in a future release
opensearch-node1       | WARNING: A terminally deprecated method in java.lang.System has been called
opensearch-node1       | WARNING: System::setSecurityManager ha

---
## 事前準備


In [3]:
%pip install -Uq opensearch-py langchain beautifulsoup4 requests boto3 python-dotenv



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import requests


In [5]:
from dotenv import load_dotenv

load_dotenv(override=True)


True

In [6]:
host = 'localhost'
# host = '172.17.0.1'
port = 9200


In [7]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    use_ssl = False,
    verify_certs = False
)

info = client.info()
print(f"Welcome to {info['version']['distribution']} {info['version']['number']}!")


Welcome to opensearch 2.11.0!


---
## 1. インデックスを作成


In [38]:
index_name = 'semantic-document-index'


In [45]:
body = {
  "settings": {
    "index.knn": True,
    "index": {
      "analysis": {
        "analyzer": {
          "custom_kuromoji_analyzer": {
            "tokenizer": "kuromoji_tokenizer",
            "filter": ["kuromoji_baseform", "ja_stop"],
            "char_filter": ["icu_normalizer"]
          }
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "content": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
    }
  }
}

response = client.indices.create(
  index_name, 
  body=body
)

print(response)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'semantic-document-index'}


---
## 2. データを登録


In [46]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader([
  "https://ja.wikipedia.org/wiki/進撃の巨人", 
  "https://ja.wikipedia.org/wiki/進撃の巨人_(アニメ)", 
  "https://ja.wikipedia.org/wiki/進撃の巨人の登場人物"])

data = loader.load()
texts = data


In [47]:
for text in texts:
  try:

    body = {
      "content": text.page_content
    }

    response = client.index(
      index=index_name,
      body=body
    )

    print(response)
  except Exception as e:
    print(e)


{'_index': 'semantic-document-index', '_id': '0p_F3YsBPf0TfMkDvhhO', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'semantic-document-index', '_id': '05_F3YsBPf0TfMkDvxiI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
{'_index': 'semantic-document-index', '_id': '1J_F3YsBPf0TfMkDwBgY', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}


---
## 1. キーワード検索


In [86]:
question = "エレンの幼馴染の名前は？"


In [87]:
body = {
    "query": {
        "match": {
            "content": {
                "query": question
            }
        }
    },
    "size": 3,
    "highlight": {
      "pre_tags": [""],
      "post_tags": [""],
      "number_of_fragments": 3,
      "fields": {
        "content": {}
      }
    }
}

res  = client.search(index=index_name,body=body)
res

{'took': 419,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 2.2016315,
  'hits': [{'_index': 'semantic-document-index',
    '_id': '1J_F3YsBPf0TfMkDwBgY',
    '_score': 2.2016315,
    '_source': {'content': '\n\n\n\n進撃の巨人の登場人物 - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nコンテンツにスキップ\n\n\n\n\n\n\n\nメインメニュー\n\n\n\n\n\nメインメニュー\nサイドバーに移動\n非表示\n\n\n\n\t\t案内\n\t\n\n\nメインページコミュニティ・ポータル最近の出来事新しいページ最近の更新おまかせ表示練習用ページアップロード (ウィキメディア・コモンズ)\n\n\n\n\n\n\t\tヘルプ\n\t\n\n\nヘルプ井戸端お知らせバグの報告寄付ウィキペディアに関するお問い合わせ\n\n\n\n\n\n言語\n\n言語間リンクはページの先頭にあるページ名の向かい側に設置されています。\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n検索\n\n\n\n\n\n\n\n\n\n\n\n検索\n\n\n\n\n\n\n\n\nアカウント作成ログイン\n\n\n\n\n\n\n個人用ツール\n\n\n\n\n\n アカウント作成 ログイン\n\n\n\n\n\n\t\tログアウトした編集者のページ もっと詳しく\n\n\n\n投稿記録トーク\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n目次\nサイドバーに移動\n非表示\n\n\n\n\nページ先頭\n\n\n\n\n\n1主要人物\n\n\n\n\n\n\n\n2第104期

In [69]:
from langchain.chat_models import BedrockChat

llm = BedrockChat(model_id="anthropic.claude-instant-v1", model_kwargs={"max_tokens_to_sample": 4000})

In [92]:
from typing import Any, Iterable, List

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever

class OpenSearchBM25Retriever(BaseRetriever):

    client: Any
    index_name: str

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:

        body = {
            "query": {
                "match": {
                    "content": {
                        "query": query
                    }
                }
            },
            "_source": False,
            "highlight": {
                "pre_tags": [""],
                "post_tags": [""],
                "fragment_size": 300,
                "number_of_fragments": 3,
                "order": "score",
                "fields": {
                    "content": {}
                }
            }
        }

        res  = self.client.search(index=index_name,body=body)

        docs = []
        for r in res["hits"]["hits"]:
            for c in r["highlight"]["content"]:
                docs.append(Document(page_content=c))
        return docs


bm25_retriever = OpenSearchBM25Retriever(client=client, index_name=index_name)

bm25_retriever.get_relevant_documents(question)


[Document(page_content='名前の由来は日露戦争時の旧日本帝国海軍旗艦の戦艦「三笠」から[10]。\nアルミン・アルレルト (Armin Arlert[1])\n声 - 井上麻里奈\n15歳→19歳。身長163cm、体重55kg（15歳時）。11月3日生まれ[3]。エレンとミカサの幼馴染で、彼らと同じくシガンシナ区で生まれ育つ。特にエレンとはミカサと知り合う以前からの親友。金髪のボブカットに茶色の眼（アニメ版では碧眼）をした少年。\n845年の巨人侵攻の翌年に敢行された領土奪還作戦で家族を失う[注 15]。大人しく内向的な性格だが芯の強さも持っており、理知的で探究心にも富む。'),
 Document(page_content='^ 名前はアニメ1話より。\n\n^ 諌山創『進撃の巨人 INSIDE 抗』（第1刷）講談社〈KCデラックス〉、2013年4月9日、44頁。ISBN\xa0978-4-06-376816-9。\xa0\n\n^ 『進撃の巨人 OUTSIDE 攻』より。\n\n^ 名前はアニメ5話より。\n\n^ a b c 名前はアニメ11話より。\n\n^ 名前はアニメ10話より。\n\n^ 名前はアニメ45話より。\n\n^ a b c 名前はアニメ25話より。\n\n^ ANSWERS 2016, p.\xa0168.\n\n^ a b ANSWERS 2016, p.\xa051.\n\n^ アニメ47話。\n\n^ 名前はアニメ2話より。\n\n^ アニメ48話。'),
 Document(page_content='エレンの幼馴染[注 6]。\n壁内人類ではほぼ絶滅したとされる東洋人[注 7]の母とアッカーマン家[注 8]の父を持つハーフ。超大型巨人が出現する1年前（844年）、希少な血筋であることを理由に3人組の強盗に目をつけられて両親を殺されるが[注 9]、エレンによって助けられ、イェーガー家に引き取られる。幼少時に母親によって一族が受け継ぐ刻印を右手首に付けられているほか[注 10]、エレンに助けられた後は彼からもらったマフラーを肌身離さず身につけている。'),
 Document(page_content='ウォール・マリア陥落（1巻）\nウォール・マリア南端より突出したシガンシナ区で生活する少年エレン・イェーガーは

In [89]:
from langchain.chains import ConversationalRetrievalChain

bm25_qa = ConversationalRetrievalChain.from_llm(
  llm, 
  retriever=bm25_retriever,
  )

bm25_response = bm25_qa({"question": question, "chat_history": [] })


In [90]:
print(bm25_response['answer'])


 エレンの幼馴染の名前はミカサ・アッカーマンです。

コンテキストからミカサ・アッカーマンがエレンの幼馴染で、イェーガー家に引き取られたことが書かれています。
